In [92]:
import zstd
import os
import sys
import json
import warnings
import numpy as np
from tqdm import tqdm
from random import shuffle

In [35]:
warnings.filterwarnings('ignore')

In [36]:
replay_dir = '../Replays/'

In [86]:
# CONSTANTS
SHIPS_PER_TEAM = 30
MAX_PLANETS = 15
AVG_TOTAL_PLANETS = 13
#AVG_MAP_SIZE_X = 48 #(384 - 288) / 2
#AVG_MAP_SIZE_Y = #

In [94]:
np.pi

3.141592653589793

In [87]:
class CyclicalList:
    """
    ReplayManager
    """
    
    


    def __init__(self, std_list):
        self.idx = 0
        self.len = len(std_list)
        self.list = std_list
        # Init constants
        self.SHIPS_PER_TEAM = 30.0
        self.MAX_PLANETS = 14.0
        self.MIN_PLANETS = 12.0
        self.AVG_TOTAL_PLANETS = 13.0
        self.MAX_SHIP_HEALTH = 255.0
        self.MAX_VELOCITY = 7.0
        
        
    def getNext(self, shuffle = False):
        self.idx = (self.idx + 1) % self.len
        if shuffle and self.idx == 0:
            shuffle(self.list)
        return self.list[self.idx]
    
    def getReplaysWithSamples(min_samples):
        """
        Iterates the buffer and gets the next replays
        that have at least min_samples overall.
        """
        cur_samples = 0
        replays = []
        for i in range(self.len):
            file_name = self.getNext(shuffle = True)
            replay = self.getReplayContent(file_name)
            replays.append(replay)
            cur_samples += replay['num_frames']
            if(cur_samples > min_samples):
                break
        return replays
    
    #def sampleFrames(self, replay_list, )
    
    def getReplayContent(self, file_name):
        with open(replay_dir + file_name, "rb") as rpfile:    
            decoded_data = zstd.decompress((rpfile.read())).decode('utf-8')
            replay_json = json.loads(decoded_data.strip())
            return replay_json
        
    def createFeatureVector(self, frame, timestep, map_width, map_height, planets, ship_id):
        # Pre-allocate matrix  
        feature_vector = np.array((6, 1))
        
        # Store team sizes
        our_team_size, their_team_size = self.getTeamSize(frame, ship_id)
        feature_vector[0] = our_team_size
        feature_vector[1] = float(our_team_size) / their_team_size
        
        # Store percentage of planets per team
        total_planets_norm, planet_balance, neutral_fraction = self.getPlanetPercentages(frame, ship_id)
        feature_vector[2] = total_planets_norm
        feature_vector[3] = planet_balance
        feature_vector[4] = neutral_fraction
        
        # Store timestep
        feature_vector[5] = timestep
        
        # Store our ship features
        magnitude, health, distance, docking_one_hot = self.getShipFeatures(frame, ship_id, map_width, map_height)
        
        # Store allied ship features
        
        # Store enemy ship features
    
    def getShipFeatures(self, frame, ship_id, map_width, map_height):
        """
        Get features for our own ship.
        """
        ships = frame['ships']
        our_team_id = self.getShipOwner(ships, ship_id)
        ship_info = ships[our_team_id][ship_id]
        
        vel_magnitude = np.sqrt(ship_info['vel_x'] ** 2 + ship_info['vel_y'] ** 2) 
        vel_magnitude_norm = (vel_magnitude - self.MAX_VELOCITY / 2) / self.MAX_VELOCITY
        health = (ship_info['health'] - MAX_SHIP_HEALTH / 2) / MAX_SHIP_HEALTH
        docking_one_hot = [1,0] if ship_info['docking']['status'] == 'undocked' else [0, 1]
        
        x_coord = ship_info['x']
        y_coord = ship_info['y']
        
        angle = np.arctan2(ship_info['vel_y'], ship_info['vel_x'])
        
        if x_coord < map_width / 2:
            # Left side
            dist = x_coord
            angle_to_side_hor = np.pi - angle
        else:
            # Right side
            dist 
            angle_to_side_hor = angle
        
        if y_coord < map_height / 2:
            # Top side
            angle_to_side_vert = np.pi/2
        else:
            # Bottom side
            angle_to_side_vert = -np.pi/2
        
        
    
    def getShipOwner(self, ships, ship_id):
        """
        Get index of ship owner
        """
        if ship_id in ships['0'].keys():
            our_team = '0'
        else:
            our_team = '1'
        return our_team
    
    def getTeamsSize(self, frame, ship_id):
        """
        Get tuple of size for both teams
        """
        ships = frame['ships']
        our_team_id = self.getShipOwner(ships, ship_id)
        their_team_id = '1' if our_team_id == '0' else '0'
        our_team_size = len(ships[our_team_id])
        their_team_size = len(ships['1'])
        return our_team_size, their_team_size
    
    def getPlanetPercentages(self, frame, ship_id):
        """
        """
        our_team_id = self.getShipOwner(frame['ships'], ship_id)
        their_team_id = '1' if our_team_id == '0' else '0'
        planets = frame['planets']
        total_planet_no = len(planets)
        owned_planets = np.sum([[1, 0] if str(planet['id']) == our_team_id else [0, 1] \
                               for planet in planets if planet['id'] is not None], axis = 0)
        our_planets_perc_norm = owned_planets[0] / total_planet_no - 0.5
        their_planets_perc_norm = owned_planets[1] / total_planet_no - 0.5
        return  our_planets_perc_norm, their_planets_perc_norm
        
        
    def getPlanetFeatures(self, frame, ship_id, max_planets):   
        """
        """
        # Get ids
        our_team_id = self.getShipOwner(frame['ships'], ship_id)
        their_team_id = '1' if our_team_id == '0' else '0'
        
        # Get numbers of planets
        planets = frame['planets']
        total_planet_no = len(planets)
        owned_planets = np.sum([[1, 0] if str(planet['id']) == our_team_id else [0, 1] \
                               for planet in planets if planet['id'] is not None], axis = 0)
        our_planets = owned_planets[0]
        their_planets = owned_planets[1]
        neutral_planets = total_planet_no - (our_planet + their_planets)
        
        balance_normalized = (our_planets - their_planets) / (2 * total_planet_no)
        neutral_fraction = neutra_planets / total_planet_no - 0.5 # Subtract 0.5 to center at 0
        
        return (total_planet_no - self.AVG_TOTAL_PLANETS) / (self.MAX_PLANETS - self.MIN_PLANETS), \
                balance_normalized, neutral_fraction

In [90]:
replay_json['frames'][0]['ships']['0']['0']

{'cooldown': 0,
 'docking': {'status': 'undocked'},
 'health': 255,
 'id': 0,
 'owner': 0,
 'vel_x': 0.0,
 'vel_y': 0.0,
 'x': 192.0,
 'y': 61.0}

In [27]:
replay_filename_list = os.listdir(replay_dir)
for file_name in os.listdir(replay_dir):
    if file_name.startswith('replay'):
        replay_filename_list.append(file_name)
replay_buffer = CyclicalList(replay_filename_list)

  4%|█▍                                    | 381/10000 [00:30<12:51, 12.47it/s]D:\Anaconda3\lib\site-packages\tqdm\_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)


In [7]:
replay_json = getReplayContent(replay_dir + replay_buffer.getNext())

In [29]:
replay_json.keys()

dict_keys(['constants', 'frames', 'height', 'map_generator', 'moves', 'num_frames', 'num_players', 'planets', 'player_names', 'poi', 'seed', 'stats', 'version', 'width'])

D:\Anaconda3\lib\site-packages\tqdm\_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)


In [43]:
replay_json['height']

224

In [89]:
replay_json['frames'][0].keys()

dict_keys(['events', 'planets', 'ships'])

In [ ]:
replay_json['num_frames']

In [ ]:
replay_json['planets'][0]

In [ ]:
for frame in replay_json['frames'][:1]:
    print(frame['ships']['1']['3'])
    print(frame['planets']['0'])

In [ ]:
for frame in replay_json['frames']:
    print(frame['events'])
replay_json['frames'][8]['events']

In [ ]:
replay_json['moves'][0]['0']

In [51]:
planets = replay_json['num_players']
print(planets)

2


In [85]:
max_ = -1000
sum_ = 0
count = 0
for i in tqdm(range(2000)):#replay_buffer.len):
    replay = replay_buffer.getNext()
    replay_json = getReplayContent(replay_dir + replay_buffer.getNext())
    if replay_json['num_players'] > 2:
        continue
    planets = replay_json['planets']
    no = len(planets)
    
    print(no)
    sum_ += no
    count += 1
    if(no > max_):
        max_ = no
print(max_)
print(sum_ / count)


  0%|                                                 | 0/2000 [00:00<?, ?it/s]

12



  0%|                                         | 1/2000 [00:00<19:17,  1.73it/s]

14
14
14



  0%|                                         | 4/2000 [00:00<06:10,  5.38it/s]

...


  2%|▊                                       | 42/2000 [00:03<02:35, 12.60it/s]

12
12



  2%|▉                                       | 44/2000 [00:03<02:39, 12.27it/s]

14
12


KeyboardInterrupt: 